In [1]:
from Isea import ScatterBrush
from IPython.display import display
from ipywidgets import Output
import pandas as pd

# sample data (note the actual column names)
df = pd.DataFrame({
    "id": [1,2,3,4],
    "x":  [1,2,3,4],
    "y":  [10,8,6,9],
    "group": ["A","B","A","B"]   # <-- categorical for color
})

# instantiate with the CORRECT column names
w = ScatterBrush(
    df,
    x="x", y="y",           # <-- must match df
    color="group",          # optional, only if it exists
    key="id",               # optional; defaults to a stable id anyway
    title="My Scatter", x_label="X", y_label="Y",
    width=720, height=420
)
display(w)

# show selection live below the chart
out = Output()
def on_sel(change):
    sel = change["new"]
    with out:
        out.clear_output()
        if sel.get("type"):
            rows = sel.get("rows", [])
            print(f"{sel['type']} selection — {len(rows)} points")
            display(pd.DataFrame(rows))
        else:
            print("Selection cleared")
w.observe(on_sel, names="selection")
display(out)


ScatterBrush(data=[{'id': 1, 'x': 1, 'y': 10, 'group': 'A'}, {'id': 2, 'x': 2, 'y': 8, 'group': 'B'}, {'id': 3…

Output()

In [2]:
display (out)

Output()